In [181]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn import svm
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import pickle


def plot_confusion_matrix(y_true, y_pred, label,
                          cmap=plt.cm.Blues):
    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data

    print(cm)

    fig, ax = plt.subplots(figsize=(10, 10))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=label, yticklabels=label,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax


import pandas as pd

In [172]:
c = [1, 10, 100, 1000]
gamma = [1e-3, 1e-4]
features = ['Date', 'Payrolls_3mo_vs_12mo', 'Effective_Fed_Funds_12_chg', 'CPI_All_Items_3_mo_annualised',
            '10Y_Treasury_Rate_12_chg', '3M_10Y_Treasury_Spread', 'S&P_500_Index_12_chg','Recession_in_6mo','Recession_in_12mo','Recession_in_24mo']
df = pd.read_csv('/Users/sebin/Desktop/companyandngo/Data/final_features.csv')[::-1]
predicted_probability=pd.DataFrame()
past_predict = pd.DataFrame()
test_prob = pd.DataFrame()
recession_data = df[features]
#df_features.join(pd.get_dummies(df['Recession']))
output_feature =['Recession_in_6mo','Recession_in_12mo','Recession_in_24mo']

In [173]:
x_training_data=recession_data[recession_data['Date'] <= '2018-02-01']
x_testing_data = recession_data[recession_data['Date'] >= '2018-02-01']


In [153]:
X_train, X_test, y_train, y_test = train_test_split(x_training_data, x_training_data.iloc[:,7:], test_size=0.30)

In [166]:
y_test

,Recession_in_6mo,Recession_in_12mo,Recession_in_24mo
373,0,0,1
109,0,0,0
397,0,0,0
106,0,0,0
311,0,0,0
...,...,...,...
690,0,0,0
390,0,0,0
424,0,0,0
731,0,0,0


In [205]:
for i in output_feature:
    svm_ac = {}
    for x, y in [(x, y) for x in c for y in gamma]:
        k = x, y
        # initalize the'model' and passing the SVM Classifier with respect to C and gamma value
        model = svm.SVC(C=x, kernel='rbf', gamma=y, probability=True,
                        tol=1e-3, random_state=123,
                        class_weight='balanced')
        accuracy = cross_val_score(model, X_train.iloc[:,1:6], y_train[i], cv=10, scoring='accuracy')
        svm_ac[k] = np.mean(accuracy)
        print("\n{:20} {:25}".format('Parameter', 'Average accuracy'))
        print('{}\t{:25}'.format(k, np.mean(accuracy)))
    print(f"\n----{i}------")
    acc_score = max(svm_ac, key=svm_ac.get)
    print(f"The best tuning parameter{acc_score}")
    svc_model = svm.SVC(C=acc_score[0], kernel='rbf', gamma=acc_score[1], probability=True,
                        tol=1e-3, random_state=123,
                        class_weight='balanced')
   
    # fitting the model with training data
    svc_model.fit(X_train.iloc[:,1:6], y_train[i])
    # predicting the model with the test data
    y_predict = svc_model.predict(X_test.iloc[:,1:6])

    # evaluating the accuracy with the ground truth and predicted value
    model_acc = accuracy_score(y_test[i], y_predict)

    print("Model Accuracy is: {}".format(model_acc))
    # classification_report((y_true, y_pred)
    print("\nPerformance table")
    print(f"\n{classification_report(y_test[i], y_predict)}")
    predicted_probability[i+"_probability"] = svc_model.predict_proba(x_testing_data.iloc[:,1:6])[:,1]
    past_predict[i+"_probability"]=svc_model.predict_proba(X_train.iloc[:,1:6])[:,1]
    test_prob[i+"_probability"]=svc_model.predict_proba(X_test.iloc[:,1:6])[:,1]
    filename = 'model'+i+'.sv'
    pickle.dump(svc_model, open(filename, 'wb'))
      


Parameter            Average accuracy         
(1, 0.001)	       0.6881349782293178

Parameter            Average accuracy         
(1, 0.0001)	      0.44034833091436865

Parameter            Average accuracy         
(10, 0.001)	       0.6841799709724238

Parameter            Average accuracy         
(10, 0.0001)	       0.6900580551523948

Parameter            Average accuracy         
(100, 0.001)	       0.6650943396226414

Parameter            Average accuracy         
(100, 0.0001)	       0.6841799709724238

Parameter            Average accuracy         
(1000, 0.001)	        0.670754716981132

Parameter            Average accuracy         
(1000, 0.0001)	       0.6650943396226414

----Recession_in_6mo------
The best tuning parameter(10, 0.0001)
Model Accuracy is: 0.6946902654867256

Performance table

              precision    recall  f1-score   support

           0       0.95      0.69      0.80       202
           1       0.22      0.71      0.33        24

    accuracy    

/Users/sebin/anaconda3/envs/p36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [175]:

predicted_probability['Date']= x_testing_data['Date'].tolist()

In [176]:
predicted_probability

,Recession_in_6mo_probability,Recession_in_12mo_probability,Recession_in_24mo_probability,Date
0,0.129073,0.135954,0.104044,2018-02-01
1,0.150927,0.143539,0.103960,2018-03-01
2,0.156422,0.168860,0.103931,2018-04-01
3,0.155328,0.179080,0.103936,2018-05-01
4,0.173881,0.209873,0.103860,2018-06-01
5,0.187439,0.199716,0.103815,2018-07-01
6,0.200844,0.231166,0.103754,2018-08-01
7,0.199201,0.247433,0.103750,2018-09-01
8,0.192119,0.240170,0.103745,2018-10-01
9,0.212733,0.261688,0.103657,2018-11-01


In [192]:
past_predict['Date']= X_train['Date'].tolist()
test_prob['Date']=X_test['Date'].tolist()


In [193]:
past_predict.append(test_prob)

,Recession_in_6mo_probability,Recession_in_12mo_probability,Recession_in_24mo_probability,Date
0,0.029583,0.014172,0.105463,1991-06-01
1,0.057241,0.028325,0.104833,2015-08-01
2,0.165625,0.159222,0.103868,1973-02-01
3,0.210281,0.196792,0.103727,1956-07-01
4,0.012034,0.004622,0.106102,1985-04-01
...,...,...,...,...
221,0.148989,0.100796,0.104059,1962-08-01
222,0.029238,0.021543,0.105391,1987-08-01
223,0.040940,0.024210,0.105162,1984-10-01
224,0.142814,0.181631,0.103920,1959-03-01


In [194]:
past_predict.append(predicted_probability)

,Recession_in_6mo_probability,Recession_in_12mo_probability,Recession_in_24mo_probability,Date
0,0.029583,0.014172,0.105463,1991-06-01
1,0.057241,0.028325,0.104833,2015-08-01
2,0.165625,0.159222,0.103868,1973-02-01
3,0.210281,0.196792,0.103727,1956-07-01
4,0.012034,0.004622,0.106102,1985-04-01
...,...,...,...,...
20,0.387151,0.170527,0.103286,2019-10-01
21,0.335060,0.144865,0.103475,2019-11-01
22,0.320268,0.154348,0.103512,2019-12-01
23,0.343812,0.164299,0.103453,2020-01-01


In [211]:
file=['modelRecession_in_6mo.sv','modelRecession_in_12mo.sv','modelRecession_in_24mo.sv']

for i in file:
    svmc = pickle.load(open(i, 'rb'))
    print(svmc.predict_proba(test.iloc[:,1:6])[:,1])
    
    



[0.09509721]
[0.04426179]
[0.10459307]


In [210]:
test=recession_data[recession_data['Date'] == '2008-02-01']
test

,Date,Payrolls_3mo_vs_12mo,Effective_Fed_Funds_12_chg,CPI_All_Items_3_mo_annualised,10Y_Treasury_Rate_12_chg,3M_10Y_Treasury_Spread,S&P_500_Index_12_chg,Recession_in_6mo,Recession_in_12mo,Recession_in_24mo
144,2008-02-01,0.995589,0.56654,1.035622,0.792373,1.62,0.945842,1,1,0
